<h1> Line of Balance </h>

<h4> By: Sean Yoon </h4>

<h3>0. Mock Data Generation</h3>

In this section, we will generate mock data for the Line of Balance (LOB) analysis. The mock data consists of 10 unique part_number and 5 unique line_number.

In [1]:
import csv, random, string
from collections import defaultdict

# ---------- Config ----------
NUM_PARTS   = 150
LINES       = [i for i in range(100, 131)]

# ---------- Part‑number helper ----------
def random_part():
    left  = ''.join(str(random.randint(1,9)) for _ in range(3))
    letter= random.choice(string.ascii_uppercase)
    mid   = ''.join(str(random.randint(1,9)) for _ in range(4))
    suf   = random.randint(1,15)
    return f"{left}{letter}{mid}-{suf}"

def make_unique_parts(n=NUM_PARTS):
    parts=set()
    while len(parts)<n:
        parts.add(random_part())
    return list(parts)

PARTS = make_unique_parts()

# ---------- Reusable qty helpers ----------
rand_demand  = lambda: random.randint(1,5)   # 1‑5 inclusive
rand_supply  = lambda lo,hi: random.randint(lo,hi)

# ---------- 1. Inventory ---------------
def generate_inventory(fn="Inventory.csv"):
    with open(fn,'w',newline='') as f:
        w=csv.writer(f); w.writerow(["part_number","qty"])
        for p in PARTS:
            w.writerow([p, rand_supply(25,45)])
    print("Generated",fn)

# ---------- 2. Company_Line_Demand  (random qty per column) ----------
def generate_boeing_line_demand(filename="Line_Demand.csv"):
    """
    For each part–line combination choose an independent random qty (1-5).
    Result: every column (line) gets its own value; rows are not constant.
    """
    rows = []
    for part in PARTS:
        for ln in LINES:
            qty = random.randint(1, 5)      # NEW: random for every column
            rows.append((part, ln, qty))

    # sort for readability: part then line
    rows.sort(key=lambda r: (r[0], r[1]))

    with open(filename, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["part_number", "line_number", "qty"])
        w.writerows(rows)

    print(f"Generated {filename}: random demand 1-5 per (part,line)")


# ---------- 3. Suppliers --------------
def generate_suppliers(fn="Supplier.csv"):
    with open(fn,'w',newline='') as f:
        csv.writer(f).writerows([
            ["supplier_id","name"],
            [1,"Supplier A"],
            [2,"Supplier B"],
            [3,"Supplier C"]
        ])
    print("Generated",fn)

# ---------- 4. Shipment Log -----------
def generate_shipment(fn="Supplier_Shipment.csv", rows=150):
    statuses = ["PACKING", "SHIPPED", "STAGED", "DELIVERED"]
    with open(fn, 'w', newline='') as f:
        w = csv.writer(f)
        w.writerow(["id","supplier_id","part_number","qty","status"])

        # --- guarantee one TRANSIT status per part ---
        idx = 1
        for part in PARTS:
            w.writerow([idx, 1, part, rand_supply(15,40),
                        random.choice(["PACKING","SHIPPED","STAGED"])])
            idx += 1

        # --- extra random rows to reach target ---
        for i in range(idx, rows+1):
            w.writerow([i, ((i-1)%3)+1, random.choice(PARTS),
                         rand_supply(15,40), random.choice(statuses)])
    print("Generated", fn)

# ---------- 5. Production Log ----------
def generate_production(fn="Supplier_Production_Status.csv", rows=150):
    statuses = ["FINAL_ASSEMBLY","SUBASSEMBLY","STAGED"]
    with open(fn, 'w', newline='') as f:
        w = csv.writer(f)
        w.writerow(["id","supplier_id","part_number","qty","status"])

        # --- guarantee one WIP status per part ---
        idx = 1
        for part in PARTS:
            w.writerow([idx, 1, part, rand_supply(15,40),
                        random.choice(["FINAL_ASSEMBLY","SUBASSEMBLY"])])
            idx += 1

        # --- extra random rows ---
        for i in range(idx, rows+1):
            w.writerow([i, ((i-1)%3)+1, random.choice(PARTS),
                         rand_supply(15,40), random.choice(statuses)])
    print("Generated", fn)

# ---------- Master runner -------------
def main():
    random.seed()                      # full randomness each run
    generate_inventory()
    generate_boeing_line_demand()
    generate_suppliers()
    generate_shipment()
    generate_production()

if __name__=="__main__":
    main()


Generated Inventory.csv
Generated Line_Demand.csv: random demand 1-5 per (part,line)
Generated Supplier.csv
Generated Supplier_Shipment.csv
Generated Supplier_Production_Status.csv


First, our line of balance will require databases from our company (Boeing) and suppliers. In the company database, we have two data tables: inventory table and demand table. Inventory table will consist of part number and quantity (qty) in stock. Demand table will consist of part number, line number, and quantity (qty) of each part required for each line. 


<h3>1. Environment Setup </h3>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

<h3>2. Data Preparation </h3>

Forming tables from the company database and supplier database, which are expected to be in CSV format, into dataframes, allowing to perform data manipulation and analysis using pandas.


In [3]:
# 1. Inventory data 
inventory_df = pd.read_csv("Inventory.csv")  # part_number | qty

# 2. Demand data per production line
demand_df = pd.read_csv("Line_Demand.csv")   # part_number | line_number | qty

# 3. Supplier data
supplier_df = pd.read_csv("Supplier.csv")    # supplier_id | name

# 4. Shipment data (used for WIP if status is PACKING or STAGED)
shipment_df = pd.read_csv("Supplier_Shipment.csv")      # id | supplier_id | part_number | qty | status

# 5. Production status log (used for WIP if status != STAGED)
production_df = pd.read_csv("Supplier_Production_Status.csv")       # id | supplier_id | part_number | qty | status


In [4]:
inventory_df.rename(columns={'qty': 'inventory'}, inplace=True)
demand_df.rename(columns={'qty': 'demand_qty'}, inplace=True)

To refrain confusion, we renamed 'qty' from inventory and demand tables to 'inventory' and 'demand_qty'.


<h3>3. Grouping 'Transit' and 'WIP'</h3>

In shipment and production data logs, we will sort statuses into 'Transit' and 'WIP' (Work In Progress) tables. Parts with 'PACKING', 'SHIPPED', and 'STAGED' are categorized as 'Transit', while 'FINAL_ASSEMBLY' and 'SUBASSEMBLY' are categorized as 'WIP'. This categorization helps in understanding the flow of parts through the production process.

In [5]:
shipment_df

,id,supplier_id,part_number,qty,status
0,1,1,719B2683-10,31,STAGED
1,2,1,665N9689-15,34,STAGED
2,3,1,657M2916-1,30,PACKING
3,4,1,581Q2939-12,24,SHIPPED
4,5,1,616Q2625-5,27,SHIPPED
...,...,...,...,...,...
145,146,1,182A6552-5,23,SHIPPED
146,147,1,885O4149-14,37,STAGED
147,148,1,294I5759-12,24,PACKING
148,149,1,763C8651-1,20,SHIPPED


In [6]:
production_df

,id,supplier_id,part_number,qty,status
0,1,1,719B2683-10,33,FINAL_ASSEMBLY
1,2,1,665N9689-15,26,FINAL_ASSEMBLY
2,3,1,657M2916-1,19,SUBASSEMBLY
3,4,1,581Q2939-12,29,FINAL_ASSEMBLY
4,5,1,616Q2625-5,15,FINAL_ASSEMBLY
...,...,...,...,...,...
145,146,1,182A6552-5,38,FINAL_ASSEMBLY
146,147,1,885O4149-14,39,FINAL_ASSEMBLY
147,148,1,294I5759-12,20,SUBASSEMBLY
148,149,1,763C8651-1,26,FINAL_ASSEMBLY


In [7]:
# Shipment
transit_df = shipment_df[shipment_df['status'] != 'DELIVERED'].groupby('part_number')['qty'].sum().reset_index(name='transit')

# Production
wip_df = production_df[production_df['status'] != 'STAGED'].groupby('part_number')['qty'].sum().reset_index(name='wip')

In [8]:
transit_df

,part_number,transit
0,114S9179-6,21
1,116M6911-8,20
2,121Q9526-15,29
3,122M5342-11,29
4,132M8365-10,40
...,...,...
145,986K6442-6,32
146,991J3459-1,34
147,996J4125-9,31
148,998R8283-7,22


In [9]:
wip_df

,part_number,wip
0,114S9179-6,39
1,116M6911-8,36
2,121Q9526-15,26
3,122M5342-11,31
4,132M8365-10,20
...,...,...
145,986K6442-6,23
146,991J3459-1,17
147,996J4125-9,27
148,998R8283-7,22


<h3>4. Merging Demand, Transit, and WIP to LoB DataFrame</h3>

In this stage, we will merge the demand, transit, and WIP dataframes into a single Line of Balance (LoB) dataframe. This will allow us to have a comprehensive view of the parts required for each line, their current status, and the quantities available.

In [10]:
# Demand
lob_df = demand_df.copy()
lob_df = lob_df.merge(inventory_df, on='part_number', how='left')
lob_df = lob_df.merge(transit_df, on='part_number', how='left')
lob_df = lob_df.merge(wip_df, on='part_number', how='left')
lob_df['inventory'] = lob_df['inventory'].fillna(0)
lob_df['transit'] = lob_df['transit'].fillna(0)
lob_df['wip'] = lob_df['wip'].fillna(0)

<h3>5. LoB Calculation & Visualization</h3>

In [11]:
# 1) initial supply
lob_df['initial_supply'] = lob_df['inventory'] + lob_df['transit'] + lob_df['wip']

# 2) sort & reset index
lob_df = lob_df.sort_values(['part_number','line_number'])
lob_df.reset_index(drop=True, inplace=True)

# 3) cumulative demand
lob_df['cum_demand'] = lob_df.groupby('part_number')['demand_qty'].cumsum()

# 4) remaining supply *before* each line
lob_df['prev_demand'] = lob_df.groupby('part_number')['cum_demand'].shift(fill_value=0)
lob_df['remaining_supply'] = lob_df['initial_supply'] - lob_df['prev_demand']

# 5) surplus *after* fulfilling this line
lob_df['surplus'] = lob_df['remaining_supply'] - lob_df['demand_qty']


In [12]:
lob_df

,part_number,line_number,demand_qty,inventory,transit,wip,initial_supply,cum_demand,prev_demand,remaining_supply,surplus
0,114S9179-6,100,1,44,21,39,104,1,0,104,103
1,114S9179-6,101,4,44,21,39,104,5,1,103,99
2,114S9179-6,102,4,44,21,39,104,9,5,99,95
3,114S9179-6,103,4,44,21,39,104,13,9,95,91
4,114S9179-6,104,2,44,21,39,104,15,13,91,89
...,...,...,...,...,...,...,...,...,...,...,...
4645,999I6798-12,126,4,38,17,31,86,74,70,16,12
4646,999I6798-12,127,4,38,17,31,86,78,74,12,8
4647,999I6798-12,128,3,38,17,31,86,81,78,8,5
4648,999I6798-12,129,5,38,17,31,86,86,81,5,0


<h4> Demand table </h4>

In [13]:
lob_df['line_number'] = lob_df['line_number'].astype(int)
pivot_demand = lob_df.pivot_table(index='part_number', columns='line_number', values='demand_qty', fill_value=0)
pivot_demand

line_number,100,101,102,103,104,105,106,107,108,109,...,121,122,123,124,125,126,127,128,129,130
part_number,,,,,,,,,,,,,,,,,,,,,
114S9179-6,1,4,4,4,2,4,3,4,5,5,...,5,3,2,3,4,4,1,3,3,2
116M6911-8,1,5,2,5,5,4,4,3,1,5,...,4,2,3,5,2,4,1,5,4,3
121Q9526-15,4,5,3,3,5,2,2,3,4,1,...,4,5,3,2,1,4,2,4,1,5
122M5342-11,2,5,2,3,2,4,1,3,5,2,...,2,5,2,2,2,4,5,5,1,4
132M8365-10,5,5,2,3,5,3,5,1,2,4,...,1,1,4,4,4,2,5,2,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986K6442-6,3,4,5,1,1,2,3,1,5,1,...,5,5,1,1,4,5,1,2,1,1
991J3459-1,5,4,4,3,2,5,1,1,2,5,...,4,2,4,3,5,1,5,1,5,5
996J4125-9,5,5,5,4,1,1,4,2,4,3,...,4,5,3,1,1,5,5,4,4,4


This is our visualization of the Line of Balance (LoB) analysis. We used matplotlib to create a color map that shows the status of each part in the production line. Green indicates that the demand of the line is met with the available inventory. Orange indicates that the demand is partially met, which the demand is greater than the inventory but could be met with the parts in transit. Yellow indicates that the demand is not met with the parts in inventory and transit but with parts in production (WIP). Red indicates that the demand is not met at all, meaning there are no parts available in inventory, transit, or WIP.

In [14]:
from pathlib import Path
from PIL import Image
import io
import ipywidgets as widgets
from IPython.display import display

# ------------------------------------------
# 1)  Collect PNG images only
# ------------------------------------------
IMG_DIR = Path("lob_diagrams")

png_files = list(IMG_DIR.glob("*.png")) + list(IMG_DIR.glob("*.PNG"))
img_files = sorted(png_files)

if not img_files:
    raise FileNotFoundError("Nothing found in 'lob_diagrams/'!")

options = {p.stem: p for p in img_files}

# ------------------------------------------
# 2)  Widgets
# ------------------------------------------
PALETTE_SIZE = 1350              #  << palette side in pixels
dd  = widgets.Dropdown(options=options, description="Choose:")
out = widgets.Output()

def make_scroll_box(path: Path) -> widgets.Box:
    """Return a scrollable square palette with a white background."""
    img = Image.open(path).convert("RGBA")         # keep alpha if it exists

    # ── sandwich on white ─────────────────────────────────────────────
    if img.mode == "RGBA":                         # only if there's transparency
        white_bg = Image.new("RGB", img.size, (255, 255, 255))
        white_bg.paste(img, mask=img.split()[3])   # 3 = alpha channel
        img = white_bg                             # now opaque RGB
    # ------------------------------------------------------------------

    buf = io.BytesIO()
    img.save(buf, format="PNG")
    buf.seek(0)
    img_widget = widgets.Image(value=buf.getvalue(), format='png')

    return widgets.Box(
        [img_widget],
        layout=widgets.Layout(
            width = f"{PALETTE_SIZE}px",
            height= f"{PALETTE_SIZE}px",
            overflow_x = "auto",
            overflow_y = "auto",
            border="1px solid #888",
            background="white"       # white palette behind the image
        )
    )


def refresh(change=None):
    out.clear_output(wait=True)
    with out:
        display(make_scroll_box(dd.value))

dd.observe(refresh, names="value")
refresh()                       # initial render
display(widgets.VBox([dd, out]))
